# Script

## Bibliotecas

In [1]:
import pandas as pd
import mysql.connector
import numpy as np
import clubes
import importlib
import os

Recarregando a informação que consta no arquivo "clubes.py".

In [2]:
importlib.reload(clubes)
print(clubes.casa)

Roma


## Partidas

### Configurando o banco de dados e carregando a consulta

In [3]:
config = {
    "host": "127.0.0.1",
    "user": "admin",
    "password": "1234",
    "database": "bet_dados"
}

In [4]:
conn = mysql.connector.connect(**config)

In [5]:
query = f"""
SELECT
	data,
  temporada,
  id_jogo,
  campeonato,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols
FROM
	partidas
WHERE
	(casa = "{clubes.casa}" OR fora = "{clubes.casa}");
"""

In [6]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [7]:
conn.close()

In [8]:
df.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2024-08-18,2024/2025,12499254,Serie A,Cagliari,0,0,Roma,3.80,3.4,2.0,Empate,0
1,2024-08-25,2024/2025,12499272,Serie A,Roma,1,2,Empoli,1.45,4.1,8.0,Empoli,3
2,2024-09-01,2024/2025,12499279,Serie A,Juventus,0,0,Roma,1.75,3.6,5.0,Empate,0
3,2024-09-15,2024/2025,12499289,Serie A,Genoa,1,1,Roma,3.30,3.3,2.2,Empate,2
4,2024-09-22,2024/2025,12499303,Serie A,Roma,3,0,Udinese,1.70,4.2,4.5,Roma,3


### Tratando as informações

### Casa Atual

Criando uma cópia do DataFrame

In [9]:
mask = (df["casa"] == clubes.casa) | (df["fora"] == clubes.casa)
casa = df.loc[mask].copy()

casa

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2024-08-18,2024/2025,12499254,Serie A,Cagliari,0,0,Roma,3.80,3.40,2.00,Empate,0
1,2024-08-25,2024/2025,12499272,Serie A,Roma,1,2,Empoli,1.45,4.10,8.00,Empoli,3
2,2024-09-01,2024/2025,12499279,Serie A,Juventus,0,0,Roma,1.75,3.60,5.00,Empate,0
3,2024-09-15,2024/2025,12499289,Serie A,Genoa,1,1,Roma,3.30,3.30,2.20,Empate,2
4,2024-09-22,2024/2025,12499303,Serie A,Roma,3,0,Udinese,1.70,4.20,4.50,Roma,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2025-09-24,2025,14572781,Liga Europa,Nice,1,2,Roma,3.15,3.25,2.30,Roma,3
273,2025-09-28,2025,13981469,Serie A,Roma,2,0,Hellas Verona,1.42,4.20,8.50,Roma,2
274,2025-10-02,2025,14572627,Liga Europa,Roma,0,1,Lille,1.70,4.00,4.50,Lille,1
275,2025-10-05,2025,13981478,Serie A,Fiorentina,1,2,Roma,2.90,3.20,2.50,Roma,3


In [10]:
casa.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2024-08-18,2024/2025,12499254,Serie A,Cagliari,0,0,Roma,3.80,3.4,2.0,Empate,0
1,2024-08-25,2024/2025,12499272,Serie A,Roma,1,2,Empoli,1.45,4.1,8.0,Empoli,3
2,2024-09-01,2024/2025,12499279,Serie A,Juventus,0,0,Roma,1.75,3.6,5.0,Empate,0
3,2024-09-15,2024/2025,12499289,Serie A,Genoa,1,1,Roma,3.30,3.3,2.2,Empate,2
4,2024-09-22,2024/2025,12499303,Serie A,Roma,3,0,Udinese,1.70,4.2,4.5,Roma,3


Selecionando a temporada atual

In [11]:
casa = casa[casa["temporada"] == "2025"]

Contando os jogos

In [12]:
jogos_casa = casa.shape[0]

jogos_casa

9

Contando as vitórias

In [13]:
vitorias = casa[casa["vencedor"] == clubes.casa]

vitorias_casa = vitorias.shape[0]

vitorias_casa

6

Contando os empates

In [14]:
empates = casa[casa["vencedor"] == "Empate"]

empates_casa = empates.shape[0]

empates_casa

0

Contando as derrotas

In [15]:
derrotas = casa[(casa["vencedor"] != clubes.casa) & (casa["vencedor"] != "Empate")]

derrotas_casa = derrotas.shape[0]

derrotas_casa

3

Gols marcados

In [16]:
gols_casa = casa.loc[casa["casa"] == clubes.casa, "gols_casa"].sum()
gols_fora = casa.loc[casa["fora"] == clubes.casa, "gols_fora"].sum()

gols_marcados_casa = gols_casa + gols_fora

gols_marcados_casa

9

Gols sofridos

In [17]:
gols_sofridos_casa = casa.loc[casa["casa"] == clubes.casa, "gols_fora"].sum()
gols_sofridos_fora = casa.loc[casa["fora"] == clubes.casa, "gols_casa"].sum()

gols_sofridos_casa = gols_sofridos_casa + gols_sofridos_fora

gols_sofridos_casa

5

Aproveitamento

In [18]:
if jogos_casa == 0:
    aproveitamento_casa = 0
else:
    aproveitamento_casa = round((vitorias_casa * 3 + empates_casa) / (jogos_casa * 3) * 100, 2)

aproveitamento_casa

66.67

Over 2.5 gols

In [19]:
over_2_casa = casa[casa["total_gols"] >= 3].shape[0]

over_2_casa

2

% de jogos com 3 ou mais gols

In [20]:
if jogos_casa == 0:
    prob_over_2_casa = 0
else:
    prob_over_2_casa = round((over_2_casa / jogos_casa) * 100, 2)

prob_over_2_casa

22.22

Under 2.5 gols

In [21]:
under_2_casa = casa[casa["total_gols"] < 3].shape[0]

under_2_casa

7

% de jogos com menos de 3 gols

In [22]:
if jogos_casa == 0:
    prob_under_2_casa = 0
else:
    prob_under_2_casa = round((under_2_casa / jogos_casa) * 100, 2)

prob_under_2_casa

77.78

Odd média

In [23]:
odd_media_casa = casa[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_fora = casa[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_casa = round((odd_media_casa + odd_media_fora) / 2, 2)

odd_media_casa

2.21

Odd média quando venceu

In [24]:
odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_vitoria_casa = round(odd_media_vitoria_casa + odd_media_vitoria_fora / 2, 2)

odd_media_vitoria_casa

C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\524038383.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\524038383.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()


2.98

Contando as goleadas

In [25]:
goleadas_mandante_casa = casa[(casa["casa"] == clubes.casa) & (casa["gols_casa"] >= 4)].shape[0]
goleadas_mandante_fora = casa[(casa["fora"] == clubes.casa) & (casa["gols_fora"] >= 4)].shape[0]

goleadas_casa = goleadas_mandante_casa + goleadas_mandante_fora

goleadas_casa

0

% de goleadas aplicadas

In [26]:
if jogos_casa == 0:
    prob_goleada_casa = 0
else:
    prob_goleada_casa = round((goleadas_casa / jogos_casa) * 100, 2)

prob_goleada_casa

0.0

Montando o DataFrame do time da casa

In [27]:
dados = {
    "Jogos": jogos_casa,
    "Vitórias": vitorias_casa,
    "Empates": empates_casa,
    "Derrotas": derrotas_casa,
    "Gols marcados": gols_marcados_casa,
    "Gols sofridos": gols_sofridos_casa,
    "Aproveitamento": aproveitamento_casa,
    "Over 2.5 gols": over_2_casa,
    "Probabilidade de over 2.5 gols": prob_over_2_casa,
    "Under 2.5 gols": under_2_casa,
    "Probabilidade de under 2.5 gols": prob_under_2_casa,
    "Odd média": odd_media_casa,
    "Odd média quando venceu": odd_media_vitoria_casa,
    "Goleadas": goleadas_casa,
    "Probabilidade de golear": prob_goleada_casa,
    "Goleadas como mandante": goleadas_mandante_casa,
    "Goleadas como visitante": goleadas_mandante_fora
}

In [28]:
ordem = [
    "Jogos", "Vitórias", "Empates", "Derrotas",
    "Gols marcados", "Gols sofridos",
    "Aproveitamento",
    "Over 2.5 gols", "Probabilidade de over 2.5 gols",
    "Under 2.5 gols", "Probabilidade de under 2.5 gols",
    "Odd média", "Odd média quando venceu",
    "Goleadas", "Probabilidade de golear",
]

faltando = [k for k in ordem if k not in dados]
if faltando:
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")


dados_ordenados = {k: dados[k] for k in ordem}
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=[clubes.casa]
)


variaveis_float = {
    "aproveitamento",
    "probabilidade de over 2.5 gols",
    "probabilidade de under 2.5 gols",
    "odd média",
    "odd média quando venceu",
    "probabilidade de golear",
}

def _fmt(nome, valor):
    if str(nome).lower() in variaveis_float:
        return f"{float(valor):.2f}"
    else:
        return f"{int(valor)}"

df_final_partidas[clubes.casa] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[clubes.casa])
]

# Índice vira coluna "Informações"
tabela_casa = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [29]:
tabela_casa.style.hide(axis="index")

Informações,Roma
Jogos,9
Vitórias,6
Empates,0
Derrotas,3
Gols marcados,9
Gols sofridos,5
Aproveitamento,66.67
Over 2.5 gols,2
Probabilidade de over 2.5 gols,22.22
Under 2.5 gols,7


In [30]:
caminho = r"C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx"

In [31]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_casa.to_excel(writer, sheet_name="Partidas Atual", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Casa (últimos 10 jogos)

In [32]:
casa = casa.sort_values("data", ascending=False).head(10)

casa

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
276,2025-10-18,2025,13981485,Serie A,Roma,0,1,Internazionale,3.50,3.25,2.15,Internazionale,1
275,2025-10-05,2025,13981478,Serie A,Fiorentina,1,2,Roma,2.90,3.20,2.50,Roma,3
274,2025-10-02,2025,14572627,Liga Europa,Roma,0,1,Lille,1.70,4.00,4.50,Lille,1
273,2025-09-28,2025,13981469,Serie A,Roma,2,0,Hellas Verona,1.42,4.20,8.50,Roma,2
272,2025-09-24,2025,14572781,Liga Europa,Nice,1,2,Roma,3.15,3.25,2.30,Roma,3
271,2025-09-21,2025,13981457,Serie A,Lazio,0,1,Roma,2.80,2.90,2.80,Roma,1
270,2025-09-14,2025,13981434,Serie A,Roma,0,1,Torino,1.57,3.70,6.50,Torino,1
269,2025-08-30,2025,13981441,Serie A,Pisa,0,1,Roma,5.25,3.50,1.73,Roma,1
268,2025-08-23,2025,13981428,Serie A,Roma,1,0,Bologna,2.20,3.10,3.55,Roma,1


Contando os jogos

In [33]:
jogos_casa = casa.shape[0]

jogos_casa

9

Contando as vitórias

In [34]:
vitorias = casa[casa["vencedor"] == clubes.casa]

vitorias_casa = vitorias.shape[0]

vitorias_casa

6

Contando os empates

In [35]:
empates = casa[casa["vencedor"] == "Empate"]

empates_casa = empates.shape[0]

empates_casa

0

Contando as derrotas

In [36]:
derrotas = casa[(casa["vencedor"] != clubes.casa) & (casa["vencedor"] != "Empate")]

derrotas_casa = derrotas.shape[0]

derrotas_casa

3

Gols marcados

In [37]:
gols_casa = casa.loc[casa["casa"] == clubes.casa, "gols_casa"].sum()
gols_fora = casa.loc[casa["fora"] == clubes.casa, "gols_fora"].sum()

gols_marcados_casa = gols_casa + gols_fora

gols_marcados_casa

9

Gols sofridos

In [38]:
gols_sofridos_casa = casa.loc[casa["casa"] == clubes.casa, "gols_fora"].sum()
gols_sofridos_fora = casa.loc[casa["fora"] == clubes.casa, "gols_casa"].sum()

gols_sofridos_casa = gols_sofridos_casa + gols_sofridos_fora

gols_sofridos_casa

5

Aproveitamento

In [39]:
aproveitamento_casa = round((vitorias_casa * 3 + empates_casa) / (jogos_casa * 3) * 100, 2)

aproveitamento_casa

66.67

Over 2.5 gols

In [40]:
over_2_casa = casa[casa["total_gols"] >= 3].shape[0]

over_2_casa

2

% de jogos com 3 ou mais gols

In [41]:
prob_over_2_casa = round((over_2_casa / jogos_casa) * 100, 2)

prob_over_2_casa

22.22

Under 2.5 gols

In [42]:
under_2_casa = casa[casa["total_gols"] < 3].shape[0]

under_2_casa

7

% de jogos com menos de 3 gols

In [43]:
prob_under_2_casa = round((under_2_casa / jogos_casa) * 100, 2)

prob_under_2_casa

77.78

Odd média

In [44]:
odd_media_casa = casa[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_fora = casa[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_casa = round((odd_media_casa + odd_media_fora) / 2, 2)

odd_media_casa

2.21

Odd média quando venceu

In [45]:
odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_vitoria_casa = round(odd_media_vitoria_casa + odd_media_vitoria_fora / 2, 2)

odd_media_vitoria_casa

C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\524038383.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\524038383.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()


2.98

Contando as goleadas

In [46]:
goleadas_mandante_casa = casa[(casa["casa"] == clubes.casa) & (casa["gols_casa"] >= 4)].shape[0]
goleadas_mandante_fora = casa[(casa["fora"] == clubes.casa) & (casa["gols_fora"] >= 4)].shape[0]

goleadas_casa = goleadas_mandante_casa + goleadas_mandante_fora

goleadas_casa

0

% de goleadas aplicadas

In [47]:
prob_goleada_casa = round((goleadas_casa / jogos_casa) * 100, 2)

prob_goleada_casa

0.0

Montando o DataFrame do time da casa

In [48]:
dados = {
    "Jogos": jogos_casa,
    "Vitórias": vitorias_casa,
    "Empates": empates_casa,
    "Derrotas": derrotas_casa,
    "Gols marcados": gols_marcados_casa,
    "Gols sofridos": gols_sofridos_casa,
    "Aproveitamento": aproveitamento_casa,
    "Over 2.5 gols": over_2_casa,
    "Probabilidade de over 2.5 gols": prob_over_2_casa,
    "Under 2.5 gols": under_2_casa,
    "Probabilidade de under 2.5 gols": prob_under_2_casa,
    "Odd média": odd_media_casa,
    "Odd média quando venceu": odd_media_vitoria_casa,
    "Goleadas": goleadas_casa,
    "Probabilidade de golear": prob_goleada_casa,
    "Goleadas como mandante": goleadas_mandante_casa,
    "Goleadas como visitante": goleadas_mandante_fora
}

In [49]:
ordem = [
    "Jogos", "Vitórias", "Empates", "Derrotas",
    "Gols marcados", "Gols sofridos",
    "Aproveitamento",
    "Over 2.5 gols", "Probabilidade de over 2.5 gols",
    "Under 2.5 gols", "Probabilidade de under 2.5 gols",
    "Odd média", "Odd média quando venceu",
    "Goleadas", "Probabilidade de golear",
]

faltando = [k for k in ordem if k not in dados]
if faltando:
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")


dados_ordenados = {k: dados[k] for k in ordem}
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=[clubes.casa]
)


variaveis_float = {
    "aproveitamento",
    "probabilidade de over 2.5 gols",
    "probabilidade de under 2.5 gols",
    "odd média",
    "odd média quando venceu",
    "probabilidade de golear",
}

def _fmt(nome, valor):
    if str(nome).lower() in variaveis_float:
        return f"{float(valor):.2f}"
    else:
        return f"{int(valor)}"

df_final_partidas[clubes.casa] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[clubes.casa])
]

# Índice vira coluna "Informações"
tabela_recente = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [50]:
tabela_recente.style.hide(axis="index")

Informações,Roma
Jogos,9
Vitórias,6
Empates,0
Derrotas,3
Gols marcados,9
Gols sofridos,5
Aproveitamento,66.67
Over 2.5 gols,2
Probabilidade de over 2.5 gols,22.22
Under 2.5 gols,7


Salvando o arquivo

In [51]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_recente.to_excel(writer, sheet_name="Desempenho 10 Jogos", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Clean sheets e clean goals (10 jogos)

In [52]:
def clean_stats_do_clube(df, clube) -> tuple[int, int]:
    # Retorna (clean_sheets, clean_goals) para 'clube'
    if clube is None or df is None or df.empty:
        return 0, 0

    m = (df["casa"] == clube) | (df["fora"] == clube)
    jogos = df.loc[m].copy()
    if jogos.empty:
        return 0, 0

    gols_favor = np.where(jogos["casa"] == clube, jogos["gols_casa"], jogos["gols_fora"])
    gols_contra = np.where(jogos["casa"] == clube, jogos["gols_fora"], jogos["gols_casa"])

    gols_favor = pd.Series(gols_favor, dtype="float").fillna(0)
    gols_contra = pd.Series(gols_contra, dtype="float").fillna(0)

    clean_sheets = int((gols_contra == 0).sum())  # não sofreu
    clean_goals  = int((gols_favor  == 0).sum())  # não marcou
    return clean_sheets, clean_goals


def montar_df_clean_unificado(df_casa, clubes, df_fora=None):
    """
    Retorna somente as estatísticas do clube mandante.
    Aceita chamada com 2 args (df_casa, clubes) ou 3 args (df_casa, df_fora, clubes).
    O df_fora é ignorado de propósito.
    """
    # Se o usuário chamar com a assinatura antiga (df_casa, df_fora, clubes), reordena:
    if df_fora is not None and not isinstance(clubes, (dict, str, type(None))):
        # Chamaram como (df_casa, df_fora, clubes) -> clubes está no 3º arg
        df_casa, df_fora, clubes = df_casa, clubes, df_fora  # swap para compatibilidade

    clube_casa = getattr(clubes, "casa", None) if not isinstance(clubes, dict) else clubes.get("casa")

    colunas_necessarias = {"casa", "fora", "gols_casa", "gols_fora"}
    faltantes = colunas_necessarias - set(df_casa.columns)
    if faltantes:
        raise KeyError(f"df_casa está sem as colunas: {sorted(faltantes)}")

    cs_casa, cg_casa = clean_stats_do_clube(df_casa, clube_casa)

    resultado = pd.DataFrame(
        [{"clube": clube_casa, "clean_sheets": cs_casa, "clean_goals": cg_casa}],
        columns=["clube", "clean_sheets", "clean_goals"],
    )
    return resultado


In [53]:
df_cleans = montar_df_clean_unificado(casa, clubes)

df_cleans

,clube,clean_sheets,clean_goals
0,Roma,4,3


### Força no mando

#### Carregando a consulta novamente

In [54]:
conn = mysql.connector.connect(**config)

In [55]:
query = f"""
SELECT
	data,
  temporada,
  id_jogo,
  campeonato,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols
FROM
	partidas
WHERE
	(casa = "{clubes.casa}" OR fora = "{clubes.casa}");
"""

In [56]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [57]:
conn.close()

In [58]:
df.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2024-08-18,2024/2025,12499254,Serie A,Cagliari,0,0,Roma,3.80,3.4,2.0,Empate,0
1,2024-08-25,2024/2025,12499272,Serie A,Roma,1,2,Empoli,1.45,4.1,8.0,Empoli,3
2,2024-09-01,2024/2025,12499279,Serie A,Juventus,0,0,Roma,1.75,3.6,5.0,Empate,0
3,2024-09-15,2024/2025,12499289,Serie A,Genoa,1,1,Roma,3.30,3.3,2.2,Empate,2
4,2024-09-22,2024/2025,12499303,Serie A,Roma,3,0,Udinese,1.70,4.2,4.5,Roma,3


#### Carregando o time da casa

In [59]:
casa = df.copy()

Selecionando o time da casa e filtrando os últimos 10 jogos

In [60]:
casa = casa[casa["casa"] == clubes.casa]

In [61]:
casa = casa.sort_values("data", ascending=False).head(10)

In [62]:
def func_desempenho_casa(casa_df):
    gc = casa_df["gols_casa"].apply(pd.to_numeric, errors="coerce").fillna(0)
    gf = casa_df["gols_fora"].apply(pd.to_numeric, errors="coerce").fillna(0)

    jogos     = int(len(casa_df))
    vitorias  = int((gc > gf).sum())
    empates   = int((gc == gf).sum())
    derrotas  = int((gc < gf).sum())
    marcados  = int(gc.sum())
    sofridos  = int(gf.sum())

    # Odds
    odd_casa = casa_df["odd_casa"].apply(pd.to_numeric, errors="coerce")
    odd_media = odd_casa.mean(skipna=True)
    odd_media_vitoria = odd_casa[gc > gf].mean(skipna=True)
    # evita NaN quando não houver vitórias
    odd_media_vitoria = 0.0 if pd.isna(odd_media_vitoria) else odd_media_vitoria

    denom = 30 if jogos >= 10 else max(jogos * 3, 1)
    aproveitamento = ((vitorias * 3 + empates) / denom) * 100          # 0–100
    media_marcados = (marcados / jogos) if jogos else 0.0
    media_sofridos = (sofridos / jogos) if jogos else 0.0

    clean_sheets = int((gf == 0).sum())
    clean_goals  = int((gc == 0).sum())

    linhas = [
        {"informacao": "Jogos",                        "dados": jogos},
        {"informacao": "Vitórias",                     "dados": vitorias},
        {"informacao": "Empates",                      "dados": empates},
        {"informacao": "Derrotas",                     "dados": derrotas},
        {"informacao": "Gols marcados",                "dados": marcados},
        {"informacao": "Gols sofridos",                "dados": sofridos},
        {"informacao": "Aproveitamento",               "dados": round(aproveitamento, 2)},
        {"informacao": "Média de gols marcados",       "dados": round(media_marcados, 2)},
        {"informacao": "Média de gols sofridos",       "dados": round(media_sofridos, 2)},
        {"informacao": "Clean sheets",                 "dados": clean_sheets},
        {"informacao": "Clean goals",                  "dados": clean_goals},
        {"informacao": "Odd média",                    "dados": round(float(odd_media) if not pd.isna(odd_media) else 0.0, 2)},
        {"informacao": "Odd média quando venceu",      "dados": round(float(odd_media_vitoria), 2)},
    ]

    df = pd.DataFrame(linhas, columns=["informacao", "dados"])

    # Inteiros obrigatórios
    int_infos = {
        "Jogos","Vitórias","Empates","Derrotas",
        "Gols marcados","Gols sofridos","Clean sheets","Clean goals"
    }
    mask_int = df["informacao"].isin(int_infos)
    df.loc[mask_int, "dados"] = df.loc[mask_int, "dados"].astype(int)

    # Formatação para exibição (2 casas só nos floats)
    float_infos = {
        "Aproveitamento", "Média de gols marcados", "Média de gols sofridos",
        "Odd média", "Odd média quando venceu"
    }
    df["dados_fmt"] = df.apply(
        lambda r: f"{r['dados']:.2f}" if r["informacao"] in float_infos else f"{int(r['dados'])}",
        axis=1
    )

    return df

In [63]:
desempenho_casa = func_desempenho_casa(casa)

desempenho_casa = desempenho_casa[["informacao", "dados_fmt"]]

desempenho_casa = desempenho_casa.rename(columns={"dados_fmt": "dados"})

desempenho_casa

,informacao,dados
0,Jogos,10
1,Vitórias,6
2,Empates,1
3,Derrotas,3
4,Gols marcados,10
5,Gols sofridos,5
6,Aproveitamento,63.33
7,Média de gols marcados,1.00
8,Média de gols sofridos,0.50
9,Clean sheets,5


### Confronto direto

#### Carregando a consulta

In [64]:
conn = mysql.connector.connect(**config)

In [65]:
query = f"""
SELECT
	data,
  temporada,
  id_jogo,
  campeonato,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols
FROM
	partidas
WHERE
	(casa = "{clubes.casa}" OR fora = "{clubes.casa}")
    AND (casa = "{clubes.fora}" OR fora = "{clubes.fora}");
"""

In [66]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [67]:
df.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols


#### H2H - Geral

In [68]:
confronto_direto = df.copy()

In [69]:
def func_desempenho_confronto_direto(confronto_direto, clubes):
    # Gols como numérico
    gc = confronto_direto["gols_casa"].apply(pd.to_numeric, errors="coerce").fillna(0)
    gf = confronto_direto["gols_fora"].apply(pd.to_numeric, errors="coerce").fillna(0)

    # Máscaras de quem é quem em cada linha
    m_casa_home = confronto_direto["casa"] == clubes.casa
    m_casa_away = confronto_direto["fora"] == clubes.casa
    m_fora_home = confronto_direto["casa"] == clubes.fora
    m_fora_away = confronto_direto["fora"] == clubes.fora

    jogos = int(len(confronto_direto))

    # Vitórias corretas (independe de onde jogou)
    vitorias_casa = int(((m_casa_home & (gc > gf)) | (m_casa_away & (gf > gc))).sum())
    empates       = int((gc == gf).sum())
    vitorias_fora = int(((m_fora_home & (gc > gf)) | (m_fora_away & (gf > gc))).sum())

    # Gols por clube (soma do lado correspondente em cada linha)
    gols_do_casa = int(gc[m_casa_home].sum() + gf[m_casa_away].sum())
    gols_do_fora = int(gc[m_fora_home].sum() + gf[m_fora_away].sum())

    # Média de gols do duelo
    if "total_gols" in confronto_direto.columns:
        total_gols = confronto_direto["total_gols"].apply(pd.to_numeric, errors="coerce")
    else:
        total_gols = gc + gf
    media_gols_duelo = round(total_gols.mean(skipna=True) if jogos else 0.0, 2)

    # ===== Odds (condicionadas ao lado) =====
    odd_casa_col = confronto_direto["odd_casa"].apply(pd.to_numeric, errors="coerce")
    odd_fora_col = confronto_direto["odd_fora"].apply(pd.to_numeric, errors="coerce")

    # Odd média do clubes.casa: quando ele é mandante usa odd_casa; quando visitante usa odd_fora
    odds_clubes_casa = pd.concat([
        odd_casa_col[m_casa_home],
        odd_fora_col[m_casa_away]
    ])
    odd_media_clubes_casa = float(odds_clubes_casa.mean(skipna=True) or 0.0)

    # Odd média do clubes.fora: análogo
    odds_clubes_fora = pd.concat([
        odd_casa_col[m_fora_home],
        odd_fora_col[m_fora_away]
    ])
    odd_media_clubes_fora = float(odds_clubes_fora.mean(skipna=True) or 0.0)

    # Odd média do vencedor (ignora empates)
    home_win = gc > gf
    away_win = gf > gc
    odds_vencedor = pd.concat([
        odd_casa_col[home_win],
        odd_fora_col[away_win]
    ])
    odd_media_vencedor = float(odds_vencedor.mean(skipna=True) or 0.0)

    # ===== Monta saída =====
    linhas = [
        {"informacao": "Jogos",                                 "dados": jogos},
        {"informacao": f"Vitórias do {clubes.casa}",            "dados": vitorias_casa},
        {"informacao": "Empates",                               "dados": empates},
        {"informacao": f"Vitórias do {clubes.fora}",            "dados": vitorias_fora},
        {"informacao": f"Gols do {clubes.casa}",                "dados": gols_do_casa},
        {"informacao": f"Gols do {clubes.fora}",                "dados": gols_do_fora},
        {"informacao": "Média de gols do duelo",                "dados": media_gols_duelo},
        {"informacao": f"Odd média do {clubes.casa}",           "dados": round(odd_media_clubes_casa, 2)},
        {"informacao": f"Odd média do {clubes.fora}",           "dados": round(odd_media_clubes_fora, 2)},
        {"informacao": "Odd média do vencedor",                 "dados": round(odd_media_vencedor, 2)},
    ]
    df = pd.DataFrame(linhas, columns=["informacao", "dados"])

    # Inteiros garantidos
    int_infos = {
        "Jogos",
        f"Vitórias do {clubes.casa}", "Empates", f"Vitórias do {clubes.fora}",
        f"Gols do {clubes.casa}", f"Gols do {clubes.fora}"
    }
    df.loc[df["informacao"].isin(int_infos), "dados"] = df.loc[df["informacao"].isin(int_infos), "dados"].astype(int)

    # Formatação de exibição: 2 casas para médias/odds
    float_infos = {
        "Média de gols do duelo",
        f"Odd média do {clubes.casa}",
        f"Odd média do {clubes.fora}",
        "Odd média do vencedor"
    }
    df["dados_fmt"] = df.apply(
        lambda r: f"{r['dados']:.2f}" if r["informacao"] in float_infos else f"{int(r['dados'])}",
        axis=1
    )

    return df

In [70]:
desempenho_confronto = func_desempenho_confronto_direto(confronto_direto, clubes)

desempenho_confronto = desempenho_confronto[["informacao", "dados_fmt"]]

desempenho_confronto = desempenho_confronto.rename(columns={"dados_fmt": "dados"})

desempenho_confronto

,informacao,dados
0,Jogos,0
1,Vitórias do Roma,0
2,Empates,0
3,Vitórias do Viktoria Plzen,0
4,Gols do Roma,0
5,Gols do Viktoria Plzen,0
6,Média de gols do duelo,0.00
7,Odd média do Roma,nan
8,Odd média do Viktoria Plzen,nan
9,Odd média do vencedor,nan


#### H2H - últimos 10 jogos

In [71]:
confronto_recente = df.copy()

In [72]:
confronto_recente = confronto_recente.sort_values("data", ascending=False).head(10)

In [73]:
def func_desempenho_confronto_recente(confronto_recente, clubes):
    # ---- Converte gols para numérico ----
    gc = confronto_recente["gols_casa"].apply(pd.to_numeric, errors="coerce").fillna(0)
    gf = confronto_recente["gols_fora"].apply(pd.to_numeric, errors="coerce").fillna(0)

    # ---- Máscaras: posição de cada clube em cada jogo ----
    m_casa_home = confronto_recente["casa"] == clubes.casa
    m_casa_away = confronto_recente["fora"] == clubes.casa
    m_fora_home = confronto_recente["casa"] == clubes.fora
    m_fora_away = confronto_recente["fora"] == clubes.fora

    jogos = int(len(confronto_recente))

    # ---- Vitórias/Empates (independe de onde jogou) ----
    vitorias_casa = int(((m_casa_home & (gc > gf)) | (m_casa_away & (gf > gc))).sum())
    empates       = int((gc == gf).sum())
    vitorias_fora = int(((m_fora_home & (gc > gf)) | (m_fora_away & (gf > gc))).sum())

    # ---- Gols por clube (lado correto em cada linha) ----
    gols_do_casa = int(gc[m_casa_home].sum() + gf[m_casa_away].sum())
    gols_do_fora = int(gc[m_fora_home].sum() + gf[m_fora_away].sum())

    # ---- Média de gols do duelo ----
    if "total_gols" in confronto_recente.columns:
        total_gols = confronto_recente["total_gols"].apply(pd.to_numeric, errors="coerce")
    else:
        total_gols = gc + gf
    media_gols_duelo = round(total_gols.mean(skipna=True) if jogos else 0.0, 2)

    # ---- Odds (usando a odd do lado correto em cada jogo) ----
    odd_casa_col = confronto_recente["odd_casa"].apply(pd.to_numeric, errors="coerce")
    odd_fora_col = confronto_recente["odd_fora"].apply(pd.to_numeric, errors="coerce")

    # Odd média do clubes.casa
    odds_clubes_casa = pd.concat([odd_casa_col[m_casa_home], odd_fora_col[m_casa_away]])
    odd_media_clubes_casa = float(odds_clubes_casa.mean(skipna=True) or 0.0)

    # Odd média do clubes.fora
    odds_clubes_fora = pd.concat([odd_casa_col[m_fora_home], odd_fora_col[m_fora_away]])
    odd_media_clubes_fora = float(odds_clubes_fora.mean(skipna=True) or 0.0)

    # Odd média do vencedor (ignora empates)
    home_win = gc > gf
    away_win = gf > gc
    odds_vencedor = pd.concat([odd_casa_col[home_win], odd_fora_col[away_win]])
    odd_media_vencedor = float(odds_vencedor.mean(skipna=True) or 0.0)

    # ---- Monta saída ----
    linhas = [
        {"informacao": "Jogos",                                 "dados": jogos},
        {"informacao": f"Vitórias do {clubes.casa}",            "dados": vitorias_casa},
        {"informacao": "Empates",                               "dados": empates},
        {"informacao": f"Vitórias do {clubes.fora}",            "dados": vitorias_fora},
        {"informacao": f"Gols do {clubes.casa}",                "dados": gols_do_casa},
        {"informacao": f"Gols do {clubes.fora}",                "dados": gols_do_fora},
        {"informacao": "Média de gols do duelo",                "dados": media_gols_duelo},
        {"informacao": f"Odd média do {clubes.casa}",           "dados": round(odd_media_clubes_casa, 2)},
        {"informacao": f"Odd média do {clubes.fora}",           "dados": round(odd_media_clubes_fora, 2)},
        {"informacao": "Odd média do vencedor",                 "dados": round(odd_media_vencedor, 2)},
    ]
    df = pd.DataFrame(linhas, columns=["informacao", "dados"])

    # Inteiros garantidos
    int_infos = {
        "Jogos",
        f"Vitórias do {clubes.casa}", "Empates", f"Vitórias do {clubes.fora}",
        f"Gols do {clubes.casa}", f"Gols do {clubes.fora}"
    }
    df.loc[df["informacao"].isin(int_infos), "dados"] = df.loc[df["informacao"].isin(int_infos), "dados"].astype(int)

    # Formatação de exibição (2 casas para médias/odds)
    float_infos = {
        "Média de gols do duelo",
        f"Odd média do {clubes.casa}",
        f"Odd média do {clubes.fora}",
        "Odd média do vencedor"
    }
    df["dados_fmt"] = df.apply(
        lambda r: f"{r['dados']:.2f}" if r["informacao"] in float_infos else f"{int(r['dados'])}",
        axis=1
    )
    return df

In [74]:
historico_recente = func_desempenho_confronto_recente(confronto_recente, clubes)

historico_recente = historico_recente[["informacao", "dados_fmt"]]

historico_recente = historico_recente.rename(columns={"dados_fmt": "dados"})

historico_recente

,informacao,dados
0,Jogos,0
1,Vitórias do Roma,0
2,Empates,0
3,Vitórias do Viktoria Plzen,0
4,Gols do Roma,0
5,Gols do Viktoria Plzen,0
6,Média de gols do duelo,0.00
7,Odd média do Roma,nan
8,Odd média do Viktoria Plzen,nan
9,Odd média do vencedor,nan


## 1º Tempo

### Carregando a consulta

In [75]:
conn = mysql.connector.connect(**config)

In [76]:
query = f"""
SELECT
    p.data,
    t1.temporada,
    t1.clube,
    CASE
        WHEN t1.clube = p.casa THEN 'Casa'
        WHEN t1.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    t1.ball_possession,
    t1.expected_goals,
    t1.big_chances,
    t1.total_shots,
    t1.shots_on_target,
    t1.shots_inside_box,
    t1.hit_woodwork,
    t1.corner_kicks,
    t1.passes,
    t1.big_chances_scored,
    t1.accurate_passes,
    t1.through_balls,
    t1.final_third_entries,
    t1.fouled_in_final_third,
    t1.dispossessed,
    t1.duels,
    t1.yellow_cards,
    t1.red_cards
FROM
    1_tempo t1
    INNER JOIN partidas p ON t1.id_jogo = p.id_jogo
WHERE
    t1.clube IN ("{clubes.casa}");
"""

In [77]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [78]:
conn.close()

In [79]:
df = df[df["temporada"] == "2025"]

In [80]:
df.head()

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,...,passes,big_chances_scored,accurate_passes,through_balls,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
265,2025-08-23,2025,Roma,Casa,54%,0.35,0,7,2,4,...,257,0,214,0,21,2,2,44%,0,0
266,2025-08-30,2025,Roma,Fora,72%,0.07,0,2,1,1,...,330,0,278,0,47,3,7,52%,1,0
267,2025-09-14,2025,Roma,Casa,70%,0.42,0,9,2,4,...,294,0,264,0,35,5,5,49%,0,0
268,2025-09-21,2025,Roma,Fora,56%,0.53,1,8,5,7,...,230,0,199,0,26,1,2,44%,0,0
269,2025-09-24,2025,Roma,Fora,67%,0.21,0,5,0,1,...,346,0,311,0,33,1,2,37%,3,0


### Casa

Jogos

In [81]:
# Filtrando apenas informações do time mandante
casa = df[df["clube"] == clubes.casa]

# Selecionando apenas os últimos 10 jogos
casa = casa.sort_values("data", ascending=False).head(10)

casa.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,...,passes,big_chances_scored,accurate_passes,through_balls,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
273,2025-10-18,2025,Roma,Casa,54%,0.26,0,3,0,2,...,264,0,230,0,14,0,5,44%,1,0
272,2025-10-05,2025,Roma,Fora,56%,1.02,1,5,2,3,...,255,0,215,0,24,0,2,52%,1,0
271,2025-10-02,2025,Roma,Casa,49%,1.22,1,10,3,7,...,247,0,196,0,25,0,5,30%,0,0


In [82]:
jogos_casa = casa.shape[0]

jogos_casa

9

Posse de bola média

In [83]:
casa["ball_possession"] = casa["ball_possession"].str.replace("%", "", regex=False)
casa["ball_possession"] = pd.to_numeric(casa["ball_possession"])

In [84]:
posse_bola_casa = round(casa["ball_possession"].mean(), 2)

posse_bola_casa

59.89

XG Médio

In [85]:
xg_medio_casa = casa.expected_goals.mean().round(2)

xg_medio_casa

0.47

XG Total

In [86]:
xg_total_casa = casa.expected_goals.sum().round(2)

xg_total_casa

4.21

Grandes chances criadas

In [87]:
grandes_chances_criadas_casa = casa.big_chances.sum()

grandes_chances_criadas_casa

3

Média de grandes chances por jogo

In [88]:
grandes_chances_jogo_casa = round(grandes_chances_criadas_casa / jogos_casa, 2)

grandes_chances_jogo_casa

0.33

Grandes chances convertidas

In [89]:
grandes_chances_convertidas_casa = casa.big_chances_scored.sum()

grandes_chances_convertidas_casa

0

Aproveitamento das grandes chances

In [90]:
aproveitamento_grandes_chances_casa = round((grandes_chances_convertidas_casa / grandes_chances_criadas_casa) * 100, 2)

aproveitamento_grandes_chances_casa

0.0

Finalizações

In [91]:
finalizacoes_casa = casa.total_shots.sum()

finalizacoes_casa

54

Média de finalizações por jogo

In [92]:
finalizacoes_jogo_casa = round(finalizacoes_casa / jogos_casa, 2)

finalizacoes_jogo_casa

6.0

Finalizações certas

In [93]:
finalizacoes_certas_casa = casa.shots_on_target.sum()

finalizacoes_certas_casa

16

Finaizações certas por jogo

In [94]:
finalizacoes_certas_jogo_casa = round(finalizacoes_certas_casa / jogos_casa, 2)

finalizacoes_certas_jogo_casa

1.78

Aproveitamento das finalizações

In [95]:
aproveitamento_finalizacoes_casa = round((finalizacoes_certas_casa / finalizacoes_casa) * 100, 2)

aproveitamento_finalizacoes_casa

29.63

Finalizações dentro da área

In [96]:
finalizacoes_area_casa = casa.shots_inside_box.sum()

finalizacoes_area_casa

32

Finalizações dentro da área por jogo

In [97]:
finalizacoes_area_jogo_casa = round(finalizacoes_area_casa / jogos_casa, 2)

finalizacoes_area_jogo_casa

3.56

Finalizações dentro da área em relação ao total de finalizações

In [98]:
perc_finalizacoes_area_casa = round((finalizacoes_area_casa / finalizacoes_casa) * 100, 2)

perc_finalizacoes_area_casa

59.26

Bolas na trave

In [99]:
bolas_trave_casa = casa.hit_woodwork.sum()

bolas_trave_casa

1

Escanteios

In [100]:
escanteios_casa = casa.corner_kicks.sum()

escanteios_casa

21

Escanteios por jogo

In [101]:
escanteios_jogo_casa = round(escanteios_casa / jogos_casa, 2)

escanteios_jogo_casa

2.33

Passes

In [102]:
passes_casa = casa.passes.sum()

passes_casa

2431

Passes certos

In [103]:
passes_certos_casa = casa.accurate_passes.sum()

passes_certos_casa

2059

Aproveitamento nos passes

In [104]:
perc_passes_certos_casa = round((passes_certos_casa / passes_casa) * 100, 2)

perc_passes_certos_casa

84.7

Entradas no terço final

In [105]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

247

Entradas no terço final por jogo

In [106]:
emtradas_terco_final_jogo_casa = round(emtradas_terco_final_casa / jogos_casa, 2)

emtradas_terco_final_jogo_casa

27.44

Faltas sofridas no terço final

In [107]:
faltas_sofridas_terco_final_casa = casa.fouled_in_final_third.sum()

faltas_sofridas_terco_final_casa

12

Perda da posse de bola

In [108]:
perda_bola_casa = casa.dispossessed.sum()

perda_bola_casa

34

Perda da posse de bola por jogo

In [109]:
perda_bola_jogo_casa = round(perda_bola_casa / jogos_casa, 2)

perda_bola_jogo_casa

3.78

Duelos ganhos

In [110]:
casa["duels"] = casa["duels"].str.replace("%", "", regex=False).astype(float)

In [111]:
duelos_ganhos_casa = casa.duels.mean().round(2)

duelos_ganhos_casa

45.56

Cartões amarelos

In [112]:
amarelos_primeiro_tempo_casa = casa.yellow_cards.sum()

amarelos_primeiro_tempo_casa

6

Cartões vermelhos

In [113]:
vermelhos_primeiro_tempo_casa = casa.red_cards.sum()

vermelhos_primeiro_tempo_casa

0

Salvando a informação

In [114]:
dados = {
    "Jogos": jogos_casa,
    "Posse de bola (média)": posse_bola_casa,
    "xG médio": xg_medio_casa,
    "xG total": xg_total_casa,
    "Grandes chances criadas": grandes_chances_criadas_casa,
    "Grandes chances por jogo": grandes_chances_jogo_casa,
    "Grandes chances convertidas": grandes_chances_convertidas_casa,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_casa,
    "Finalizações": finalizacoes_casa,
    "Finalizações por jogo": finalizacoes_jogo_casa,
    "Finalizações certas": finalizacoes_certas_casa,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_casa,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_casa,
    "Finalizações dentro da área": finalizacoes_area_casa,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_casa,
    "% de finalizações dentro da área": perc_finalizacoes_area_casa,
    "Bolas na trave": bolas_trave_casa,
    "Escanteios": escanteios_casa,
    "Escanteios por jogo": escanteios_jogo_casa,
    "Passes": passes_casa,
    "Passes certos": passes_certos_casa,
    "Assertividade nos passes": perc_passes_certos_casa,
    "Entradas no terço final": emtradas_terco_final_casa,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_casa,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_casa,
    "Perdas de bola": perda_bola_casa,
    "Perdas de bola por jogo": perda_bola_jogo_casa,
    "Duelos ganhos (média)": duelos_ganhos_casa,
    "Cartões amarelos": amarelos_primeiro_tempo_casa,
    "Cartões vermelhos": vermelhos_primeiro_tempo_casa
}

In [115]:
ordem = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [116]:
import unicodedata

def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados = {normalizar(k): v for k, v in dados.items()}
ordem = [normalizar(k) for k in ordem]


In [117]:
# --- Checar chaves faltantes ---
faltando = [k for k in ordem if k not in dados]
if faltando:
    print("Debug - Chaves em dados:", list(dados.keys()))
    print("Debug - Chaves em ordem:", ordem)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")

# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Reordenar o dicionário já com valores escalares ---
dados_ordenados = {k: extrair_valor(dados[k]) for k in ordem}

# --- Criar DataFrame ---
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=["_temp"]
)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)  # garante escalar
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        # se não der para converter em número, retorna como string
        return str(valor)

# --- Nome do clube ---
coluna_clube = clubes.casa

# Renomeia a coluna temporária para o nome do clube
df_final_partidas = df_final_partidas.rename(columns={"_temp": coluna_clube})

# Aplicar formatação
df_final_partidas[coluna_clube] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[coluna_clube])
]

# Índice vira coluna "Informações"
tabela_tempo1 = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [118]:
tabela_tempo1.style.hide(axis="index")

Informações,Roma
Jogos,9
Posse de bola (média),59.89
xG médio,0.47
xG total,4.21
Grandes chances criadas,3
Grandes chances por jogo,0
Grandes chances convertidas,0
Aproveitamento das grandes chances,0.00
Finalizações,54
Finalizações por jogo,6.00


Salvando o arquivo no Excel

In [119]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_tempo1.to_excel(writer, sheet_name="Tempo 1", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


## 2º Tempo

### Carregando a consulta

In [120]:
conn = mysql.connector.connect(**config)

In [121]:
query = f"""
SELECT
    p.data,
    t2.temporada,
    t2.clube,
    CASE
        WHEN t2.clube = p.casa THEN 'Casa'
        WHEN t2.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    t2.ball_possession,
    t2.expected_goals,
    t2.big_chances,
    t2.total_shots,
    t2.shots_on_target,
    t2.shots_inside_box,
    t2.hit_woodwork,
    t2.corner_kicks,
    t2.passes,
    t2.big_chances_scored,
    t2.accurate_passes,
    t2.final_third_entries,
    t2.fouled_in_final_third,
    t2.dispossessed,
    t2.duels,
    t2.yellow_cards,
    t2.red_cards
FROM
    2_tempo t2
    INNER JOIN partidas p ON t2.id_jogo = p.id_jogo
WHERE
    t2.clube IN ("{clubes.casa}");
"""

In [122]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [123]:
conn.close()

In [124]:
df = df[df["temporada"] == "2025"]

In [125]:
df.head()

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,...,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
265,2025-08-23,2025,Roma,Casa,46%,1.22,2,7,2,4,...,2,157,1,124,21,1,5,50%,1,0
266,2025-08-30,2025,Roma,Fora,65%,0.71,1,8,4,5,...,5,251,0,208,33,2,6,58%,0,0
267,2025-09-14,2025,Roma,Casa,76%,0.72,0,13,3,8,...,4,296,0,266,37,2,5,45%,2,0
268,2025-09-21,2025,Roma,Fora,46%,0.63,0,5,1,5,...,4,178,0,137,17,1,8,43%,1,0
269,2025-09-24,2025,Roma,Fora,47%,0.59,1,7,4,5,...,2,227,1,180,30,0,3,44%,2,0


### Casa

Jogos

In [126]:
# Selecionando apenas o time mandante
casa = df[df["clube"] == clubes.casa]

# Filtrando apenas os últimos 10 jogos
casa = casa.sort_values("data", ascending=False).head(10)

casa.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,...,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
273,2025-10-18,2025,Roma,Casa,61%,1.09,3,12,5,9,...,2,245,0,214,22,4,6,52%,3,0
272,2025-10-05,2025,Roma,Fora,54%,0.33,1,3,1,2,...,1,212,0,169,24,2,5,46%,2,0
271,2025-10-02,2025,Roma,Casa,51%,1.39,3,10,3,6,...,4,212,0,158,30,2,7,41%,0,0


In [127]:
jogos_casa = casa.shape[0]

jogos_casa

9

Posse de bola média

In [128]:
casa["ball_possession"] = casa["ball_possession"].str.replace("%", "", regex=False)
casa["ball_possession"] = pd.to_numeric(casa["ball_possession"])

In [129]:
posse_bola_casa = casa.ball_possession.mean().round(2)

posse_bola_casa

55.67

XG Médio

In [130]:
xg_medio_casa = casa.expected_goals.mean().round(2)

xg_medio_casa

0.83

XG Total

In [131]:
xg_total_casa = casa.expected_goals.sum().round(2)

xg_total_casa

7.5

Grandes chances criadas

In [132]:
grandes_chances_criadas_casa = casa.big_chances.sum()

grandes_chances_criadas_casa

13

Média de grandes chances por jogo

In [133]:
grandes_chances_jogo_casa = round(grandes_chances_criadas_casa / jogos_casa, 2)

grandes_chances_jogo_casa

1.44

Grandes chances convertidas

In [134]:
grandes_chances_convertidas_casa = casa.big_chances_scored.sum()

grandes_chances_convertidas_casa

3

Aproveitamento das grandes chances

In [135]:
aproveitamento_grandes_chances_casa = round((grandes_chances_convertidas_casa / grandes_chances_criadas_casa) * 100, 2)

aproveitamento_grandes_chances_casa

23.08

Finalizações

In [136]:
finalizacoes_casa = casa.total_shots.sum()

finalizacoes_casa

70

Média de finalizações por jogo

In [137]:
finalizacoes_jogo_casa = round(finalizacoes_casa / jogos_casa, 2)

finalizacoes_jogo_casa

7.78

Finalizações certas

In [138]:
finalizacoes_certas_casa = casa.shots_on_target.sum()

finalizacoes_certas_casa

27

Finaizações certas por jogo

In [139]:
finalizacoes_certas_jogo_casa = round(finalizacoes_certas_casa / jogos_casa, 2)

finalizacoes_certas_jogo_casa

3.0

Aproveitamento das finalizações

In [140]:
aproveitamento_finalizacoes_casa = round((finalizacoes_certas_casa / finalizacoes_casa) * 100, 2)

aproveitamento_finalizacoes_casa

38.57

Finalizações dentro da área

In [141]:
finalizacoes_area_casa = casa.shots_inside_box.sum()

finalizacoes_area_casa

47

Finalizações dentro da área por jogo

In [142]:
finalizacoes_area_jogo_casa = round(finalizacoes_area_casa / jogos_casa, 2)

finalizacoes_area_jogo_casa

5.22

Finalizações dentro da área em relação ao total de finalizações

In [143]:
perc_finalizacoes_area_casa = round((finalizacoes_area_casa / finalizacoes_casa) * 100, 2)

perc_finalizacoes_area_casa

67.14

Bolas na trave

In [144]:
bolas_trave_casa = casa.hit_woodwork.sum()

bolas_trave_casa

0

Escanteios

In [145]:
escanteios_casa = casa.corner_kicks.sum()

escanteios_casa

28

Escanteios por jogo

In [146]:
escanteios_jogo_casa = round(escanteios_casa / jogos_casa, 2)

escanteios_jogo_casa

3.11

Passes

In [147]:
passes_casa = casa.passes.sum()

passes_casa

2002

Passes certos

In [148]:
passes_certos_casa = casa.accurate_passes.sum()

passes_certos_casa

1619

Aproveitamento nos passes

In [149]:
perc_passes_certos_casa = round((passes_certos_casa / passes_casa) * 100, 2)

perc_passes_certos_casa

80.87

Entradas no terço final

In [150]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

241

Entradas no terço final por jogo

In [151]:
emtradas_terco_final_jogo_casa = round(emtradas_terco_final_casa / jogos_casa, 2)

emtradas_terco_final_jogo_casa

26.78

Faltas sofridas no terço final

In [152]:
faltas_sofridas_terco_final_casa = casa.fouled_in_final_third.sum()

faltas_sofridas_terco_final_casa

17

Perda da posse de bola

In [153]:
perda_bola_casa = casa.dispossessed.sum()

perda_bola_casa

50

Perda da posse de bola por jogo

In [154]:
perda_bola_jogo_casa = round(perda_bola_casa / jogos_casa, 2)

perda_bola_jogo_casa

5.56

Duelos ganhos

In [155]:
casa["duels"] = casa["duels"].str.replace("%", "", regex=False).astype(float)

In [156]:
duelos_ganhos_casa = casa.duels.mean().round(2)

duelos_ganhos_casa

46.67

Cartões amarelos

In [157]:
amarelos_segundo_tempo_casa = casa.yellow_cards.sum()

amarelos_segundo_tempo_casa

11

Cartões vermelhos

In [158]:
vermelhos_segundo_tempo_casa = casa.red_cards.sum()

vermelhos_segundo_tempo_casa

0

Salvando a informação

In [159]:
dados = {
    "Jogos": jogos_casa,
    "Posse de bola (média)": posse_bola_casa,
    "xG médio": xg_medio_casa,
    "xG total": xg_total_casa,
    "Grandes chances criadas": grandes_chances_criadas_casa,
    "Grandes chances por jogo": grandes_chances_jogo_casa,
    "Grandes chances convertidas": grandes_chances_convertidas_casa,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_casa,
    "Finalizações": finalizacoes_casa,
    "Finalizações por jogo": finalizacoes_jogo_casa,
    "Finalizações certas": finalizacoes_certas_casa,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_casa,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_casa,
    "Finalizações dentro da área": finalizacoes_area_casa,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_casa,
    "% de finalizações dentro da área": perc_finalizacoes_area_casa,
    "Bolas na trave": bolas_trave_casa,
    "Escanteios": escanteios_casa,
    "Escanteios por jogo": escanteios_jogo_casa,
    "Passes": passes_casa,
    "Passes certos": passes_certos_casa,
    "Assertividade nos passes": perc_passes_certos_casa,
    "Entradas no terço final": emtradas_terco_final_casa,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_casa,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_casa,
    "Perdas de bola": perda_bola_casa,
    "Perdas de bola por jogo": perda_bola_jogo_casa,
    "Duelos ganhos (média)": duelos_ganhos_casa,
    "Cartões amarelos": amarelos_segundo_tempo_casa,
    "Cartões vermelhos": vermelhos_segundo_tempo_casa
}

In [160]:
ordem = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [161]:
import unicodedata

def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados = {normalizar(k): v for k, v in dados.items()}
ordem = [normalizar(k) for k in ordem]


In [162]:
# --- Checar chaves faltantes ---
faltando = [k for k in ordem if k not in dados]
if faltando:
    print("Debug - Chaves em dados:", list(dados.keys()))
    print("Debug - Chaves em ordem:", ordem)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")

# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Reordenar o dicionário já com valores escalares ---
dados_ordenados = {k: extrair_valor(dados[k]) for k in ordem}

# --- Criar DataFrame ---
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=["_temp"]
)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)  # garante escalar
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        # se não der para converter em número, retorna como string
        return str(valor)

# --- Nome do clube ---
coluna_clube = clubes.casa

# Renomeia a coluna temporária para o nome do clube
df_final_partidas = df_final_partidas.rename(columns={"_temp": coluna_clube})

# Aplicar formatação
df_final_partidas[coluna_clube] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[coluna_clube])
]

# Índice vira coluna "Informações"
tabela_tempo2 = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [163]:
tabela_tempo2.style.hide(axis="index")

Informações,Roma
Jogos,9
Posse de bola (média),55.67
xG médio,0.83
xG total,7.50
Grandes chances criadas,13
Grandes chances por jogo,1
Grandes chances convertidas,3
Aproveitamento das grandes chances,23.08
Finalizações,70
Finalizações por jogo,7.78


Salvando o arquivo no Excel

In [164]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_tempo2.to_excel(writer, sheet_name="Tempo 2", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


## Partida completa

### Carregando a consulta

In [165]:
conn = mysql.connector.connect(**config)

In [166]:
query = f"""
SELECT
    p.data,
    ep.temporada,
    ep.clube,
    CASE
        WHEN ep.clube = p.casa THEN 'Casa'
        WHEN ep.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    ep.ball_possession,
    ep.expected_goals,
    ep.big_chances,
    ep.total_shots,
    ep.shots_on_target,
    ep.shots_inside_box,
    ep.hit_woodwork,
    ep.corner_kicks,
    ep.passes,
    ep.big_chances_scored,
    ep.accurate_passes,
    ep.final_third_entries,
    ep.fouled_in_final_third,
    ep.dispossessed,
    ep.duels,
    ep.yellow_cards
FROM
    estatisticas_partidas ep
    INNER JOIN partidas p ON ep.id_jogo = p.id_jogo
WHERE
    ep.clube IN ("{clubes.casa}");
"""

In [167]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [168]:
conn.close()

In [169]:
df = df[df["temporada"] == "2025"]

In [170]:
df.head()

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards
266,2025-08-23,2025,Roma,Casa,51%,1.56,2,14,4,8,1,2,414,1,338,42,3,7,47%,1
267,2025-08-30,2025,Roma,Fora,69%,0.78,1,10,5,6,0,7,581,0,486,80,5,13,55%,1
268,2025-09-14,2025,Roma,Casa,73%,1.14,0,22,5,12,0,7,590,0,530,72,7,10,47%,2
269,2025-09-21,2025,Roma,Fora,51%,1.16,1,13,6,12,0,6,408,0,336,43,2,10,43%,1
270,2025-09-24,2025,Roma,Fora,57%,0.80,1,12,4,6,0,5,573,1,491,63,1,5,41%,5


### Casa

In [171]:
# Selecionando apenas o time de casa
casa = df[df["clube"] == clubes.casa]

# Filtrando apenas os últimos 10 jogos
casa = casa.sort_values("data", ascending=False).head(10)

casa.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards
274,2025-10-18,2025,Roma,Casa,57%,1.35,3,15,5,11,0,4,509,0,444,36,4,11,49%,4
273,2025-10-05,2025,Roma,Fora,55%,1.35,2,8,3,5,0,4,467,0,384,48,2,7,49%,3
272,2025-10-02,2025,Roma,Casa,50%,2.61,4,20,6,13,0,6,459,0,354,55,2,12,35%,0


In [172]:
jogos_casa = casa.shape[0]

jogos_casa

9

Posse de bola média

In [173]:
casa["ball_possession"] = casa["ball_possession"].str.replace("%", "", regex=False)
casa["ball_possession"] = pd.to_numeric(casa["ball_possession"])

In [174]:
posse_bola_casa = casa.ball_possession.mean().round(2)

posse_bola_casa

57.78

XG Médio

In [175]:
xg_medio_casa = casa.expected_goals.mean().round(2)

xg_medio_casa

1.3

XG Total

In [176]:
xg_total_casa = casa.expected_goals.sum().round(2)

xg_total_casa

11.7

Grandes chances criadas

In [177]:
grandes_chances_criadas_casa = casa.big_chances.sum()

grandes_chances_criadas_casa

16

Média de grandes chances por jogo

In [178]:
grandes_chances_jogo_casa = round(grandes_chances_criadas_casa / jogos_casa, 2)

grandes_chances_jogo_casa

1.78

Grandes chances convertidas

In [179]:
grandes_chances_convertidas_casa = casa.big_chances_scored.sum()

grandes_chances_convertidas_casa

3

Aproveitamento das grandes chances

In [180]:
aproveitamento_grandes_chances_casa = round((grandes_chances_convertidas_casa / grandes_chances_criadas_casa) * 100, 2)

aproveitamento_grandes_chances_casa

18.75

Finalizações


In [181]:
finalizacoes_casa = casa.total_shots.sum()

finalizacoes_casa

124

Média de finalizações por jogo

In [182]:
finalizacoes_jogo_casa = round(finalizacoes_casa / jogos_casa, 2)

finalizacoes_jogo_casa

13.78

Finalizações certas

In [183]:
finalizacoes_certas_casa = casa.shots_on_target.sum()

finalizacoes_certas_casa

43

Finaizações certas por jogo

In [184]:
finalizacoes_certas_jogo_casa = round(finalizacoes_certas_casa / jogos_casa, 2)

finalizacoes_certas_jogo_casa

4.78

Aproveitamento das finalizações

In [185]:
aproveitamento_finalizacoes_casa = round((finalizacoes_certas_casa / finalizacoes_casa) * 100, 2)

aproveitamento_finalizacoes_casa

34.68

Finalizações dentro da área

In [186]:
finalizacoes_area_casa = casa.shots_inside_box.sum()

finalizacoes_area_casa

79

Finalizações dentro da área por jogo

In [187]:
finalizacoes_area_jogo_casa = round(finalizacoes_area_casa / jogos_casa, 2)

finalizacoes_area_jogo_casa

8.78

Finalizações dentro da área em relação ao total de finalizações

In [188]:
perc_finalizacoes_area_casa = round((finalizacoes_area_casa / finalizacoes_casa) * 100, 2)

perc_finalizacoes_area_casa

63.71

Bolas na trave

In [189]:
bolas_trave_casa = casa.hit_woodwork.sum()

bolas_trave_casa

1

Escanteios

In [190]:
escanteios_casa = casa.corner_kicks.sum()

escanteios_casa

49

Escanteios por jogo

In [191]:
escanteios_jogo_casa = round(escanteios_casa / jogos_casa, 2)

escanteios_jogo_casa

5.44

Passes

In [192]:
passes_casa = casa.passes.sum()

passes_casa

4433

Passes certos

In [193]:
passes_certos_casa = casa.accurate_passes.sum()

passes_certos_casa

3678

Aproveitamento nos passes

In [194]:
perc_passes_certos_casa = round((passes_certos_casa / passes_casa) * 100, 2)

perc_passes_certos_casa

82.97

Entradas no terço final

In [195]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

488

Entradas no terço final

In [196]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

488

Entradas no terço final por jogo

In [197]:
emtradas_terco_final_jogo_casa = round(emtradas_terco_final_casa / jogos_casa, 2)

emtradas_terco_final_jogo_casa

54.22

Faltas sofridas no terço final

In [198]:
faltas_sofridas_terco_final_casa = casa.fouled_in_final_third.sum()

faltas_sofridas_terco_final_casa

29

Perda da posse de bola

In [199]:
perda_bola_casa = casa.dispossessed.sum()

perda_bola_casa

84

Perda da posse de bola por jogo

In [200]:
perda_bola_jogo_casa = round(perda_bola_casa / jogos_casa, 2)

perda_bola_jogo_casa

9.33

Duelos ganhos

In [201]:
casa["duels"] = casa["duels"].str.replace("%", "", regex=False).astype(float)

In [202]:
duelos_ganhos_casa = casa.duels.mean().round(2)

duelos_ganhos_casa

46.0

Cartões amarelos

In [203]:
amarelos_casa = casa.yellow_cards.sum()

amarelos_casa

17

Cartões vermelhos

In [204]:
vermelhos_casa = vermelhos_primeiro_tempo_casa + vermelhos_segundo_tempo_casa

vermelhos_casa

0

Salvando a informação

In [205]:
dados = {
    "Jogos": jogos_casa,
    "Posse de bola (média)": posse_bola_casa,
    "xG médio": xg_medio_casa,
    "xG total": xg_total_casa,
    "Grandes chances criadas": grandes_chances_criadas_casa,
    "Grandes chances por jogo": grandes_chances_jogo_casa,
    "Grandes chances convertidas": grandes_chances_convertidas_casa,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_casa,
    "Finalizações": finalizacoes_casa,
    "Finalizações por jogo": finalizacoes_jogo_casa,
    "Finalizações certas": finalizacoes_certas_casa,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_casa,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_casa,
    "Finalizações dentro da área": finalizacoes_area_casa,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_casa,
    "% de finalizações dentro da área": perc_finalizacoes_area_casa,
    "Bolas na trave": bolas_trave_casa,
    "Escanteios": escanteios_casa,
    "Escanteios por jogo": escanteios_jogo_casa,
    "Passes": passes_casa,
    "Passes certos": passes_certos_casa,
    "Assertividade nos passes": perc_passes_certos_casa,
    "Entradas no terço final": emtradas_terco_final_casa,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_casa,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_casa,
    "Perdas de bola": perda_bola_casa,
    "Perdas de bola por jogo": perda_bola_jogo_casa,
    "Duelos ganhos (média)": duelos_ganhos_casa,
    "Cartões amarelos": amarelos_casa,
    "Cartões vermelhos": vermelhos_casa
}

In [206]:
ordem = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [207]:
import unicodedata

def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados = {normalizar(k): v for k, v in dados.items()}
ordem = [normalizar(k) for k in ordem]


In [208]:
# --- Checar chaves faltantes ---
faltando = [k for k in ordem if k not in dados]
if faltando:
    print("Debug - Chaves em dados:", list(dados.keys()))
    print("Debug - Chaves em ordem:", ordem)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")

# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Reordenar o dicionário já com valores escalares ---
dados_ordenados = {k: extrair_valor(dados[k]) for k in ordem}

# --- Criar DataFrame ---
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=["_temp"]
)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)  # garante escalar
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        # se não der para converter em número, retorna como string
        return str(valor)

# --- Nome do clube ---
coluna_clube = clubes.casa

# Renomeia a coluna temporária para o nome do clube
df_final_partidas = df_final_partidas.rename(columns={"_temp": coluna_clube})

# Aplicar formatação
df_final_partidas[coluna_clube] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[coluna_clube])
]

# Índice vira coluna "Informações"
tabela_completo = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [209]:
tabela_completo.style.hide(axis="index")

Informações,Roma
Jogos,9
Posse de bola (média),57.78
xG médio,1.30
xG total,11.70
Grandes chances criadas,16
Grandes chances por jogo,1
Grandes chances convertidas,3
Aproveitamento das grandes chances,18.75
Finalizações,124
Finalizações por jogo,13.78


Salvando o arquivo no Excel

In [210]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_completo.to_excel(writer, sheet_name="Estatísticas Partidas", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


## Jogadores

### Configurando o banco de dados e carregando a consulta

In [211]:
conn = mysql.connector.connect(**config)

In [212]:
query = f"""
SELECT
    p.data,
    p.id_jogo,
    ej.temporada,
    CASE
        WHEN ej.team = p.casa THEN 'Casa'
        WHEN ej.team = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    ej.player_id,
    ej.name,
    CASE
        WHEN ej.position = 'G' THEN 'Goleiro'
        WHEN ej.position = 'D' THEN 'Defensor'
        WHEN ej.position = 'M' THEN 'Meio-campo'
        WHEN ej.position = 'F' THEN 'Atacante'
        ELSE 'Verificar'
    END AS position,
    ej.team,
    ej.rating,
    ej.minutes_played,
    ej.goals,
    ej.goal_assist,
    ej.big_chance_created,
    ej.key_pass,
    ej.expected_goals
FROM
    estatisticas_jogadores ej
    INNER JOIN partidas p ON ej.id_jogo = p.id_jogo
WHERE
    ej.minutes_played > 0
    AND team IN ('{clubes.casa}');
"""

In [213]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_17424\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [214]:
conn.close()

### Mais minutos

Criando uma cópia do DataFrame original

In [215]:
# Filtrando apenas a temporada atual
df = df[df["temporada"] == "2025"]

df.head(3)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals
2931,2025-08-30,13981441,2025,Fora,1128530,Neil El Aynaoui,Meio-campo,Roma,0.0,1,0,0,0,0,0.0
2932,2025-08-30,13981441,2025,Fora,982778,Devyne Rensch,Defensor,Roma,0.0,1,0,0,0,0,0.0
2933,2025-08-30,13981441,2025,Fora,893008,Zeki Çelik,Defensor,Roma,6.9,8,0,0,0,0,0.0


In [216]:
# Criando a cópia do DataFrame
mais_minutos = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time+jogo"

In [217]:
game_dates = (
    mais_minutos
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [218]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [219]:
mais_minutos = (
    mais_minutos
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores e somando os minutos

In [220]:
mais_minutos = (
    mais_minutos
      .groupby(["player_id", "name", "position", "team"], as_index=False)["minutes_played"]
      .sum()
      .rename(columns={"minutes_played": "minutos"})
      .sort_values(["team", "minutos"], ascending=[True, False])
      .reset_index(drop=True))

In [221]:
mais_minutos.head(3)

,player_id,name,position,team,minutos
0,793986,Mile Svilar,Goleiro,Roma,810
1,848287,Evan Ndicka,Defensor,Roma,756
2,611210,Gianluca Mancini,Defensor,Roma,755


Montando o top 5 de cada clube

In [222]:
mais_minutos_casa = mais_minutos[mais_minutos["team"] == clubes.casa].head(5)

mais_minutos_casa

,player_id,name,position,team,minutos
0,793986,Mile Svilar,Goleiro,Roma,810
1,848287,Evan Ndicka,Defensor,Roma,756
2,611210,Gianluca Mancini,Defensor,Roma,755
3,1082406,Matías Soulé,Atacante,Roma,728
4,974087,Manu Koné,Meio-campo,Roma,720


In [223]:
mais_minutos_fora = mais_minutos[mais_minutos["team"] == clubes.fora].head(5)

mais_minutos_fora

,player_id,name,position,team,minutos


Unificando as informações em um único DataFrame

In [224]:
mais_minutos = pd.concat([mais_minutos_casa, mais_minutos_fora], ignore_index=True).sort_values("minutos", ascending=False)

mais_minutos

,player_id,name,position,team,minutos
0,793986,Mile Svilar,Goleiro,Roma,810
1,848287,Evan Ndicka,Defensor,Roma,756
2,611210,Gianluca Mancini,Defensor,Roma,755
3,1082406,Matías Soulé,Atacante,Roma,728
4,974087,Manu Koné,Meio-campo,Roma,720


### Resultado: mais minutos

Eliminando a primeira coluna

In [225]:
mais_minutos = mais_minutos.drop(mais_minutos.columns[0], axis=1)

mais_minutos

,name,position,team,minutos
0,Mile Svilar,Goleiro,Roma,810
1,Evan Ndicka,Defensor,Roma,756
2,Gianluca Mancini,Defensor,Roma,755
3,Matías Soulé,Atacante,Roma,728
4,Manu Koné,Meio-campo,Roma,720


Salvando no Excel

In [226]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    mais_minutos.head(10).to_excel(writer, sheet_name="Mais Minutos", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Artilharia

In [227]:
gols = df.copy()

Selecionando as colunas de trabalho

In [228]:
gols = gols[["data", "id_jogo", "player_id", "name", "position", "team", "goals"]]

gols.head()

,data,id_jogo,player_id,name,position,team,goals
2931,2025-08-30,13981441,1128530,Neil El Aynaoui,Meio-campo,Roma,0
2932,2025-08-30,13981441,982778,Devyne Rensch,Defensor,Roma,0
2933,2025-08-30,13981441,893008,Zeki Çelik,Defensor,Roma,0
2934,2025-08-30,13981441,798820,Artem Dovbyk,Atacante,Roma,0
2935,2025-08-30,13981441,256219,Paulo Dybala,Atacante,Roma,0


Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [229]:
game_dates = (
    gols
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [230]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [231]:
gols = (
    gols
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores e somando os minutos

In [232]:
gols = (
    gols
      .groupby(["player_id", "name", "position", "team"], as_index=False)["goals"]
      .sum()
      .rename(columns={"goals": "gols"})
      .sort_values(["team", "gols"], ascending=[True, False])
      .reset_index(drop=True))

gols.head(3)

,player_id,name,position,team,gols
0,1082406,Matías Soulé,Atacante,Roma,3
1,186855,Bryan Cristante,Meio-campo,Roma,1
2,555540,Lorenzo Pellegrini,Meio-campo,Roma,1


In [233]:
gols = gols[gols["gols"] > 0]

gols.head(3)

,player_id,name,position,team,gols
0,1082406,Matías Soulé,Atacante,Roma,3
1,186855,Bryan Cristante,Meio-campo,Roma,1
2,555540,Lorenzo Pellegrini,Meio-campo,Roma,1


Removendo os valores duplicados

In [234]:
gols = gols.drop_duplicates()

### Resultado: Artilharia

Ordenando do maior para o menor valor

In [235]:
gols = gols.sort_values("gols", ascending=False)

gols = gols.head(10)

gols

,player_id,name,position,team,gols
0,1082406,Matías Soulé,Atacante,Roma,3
1,186855,Bryan Cristante,Meio-campo,Roma,1
2,555540,Lorenzo Pellegrini,Meio-campo,Roma,1
3,611210,Gianluca Mancini,Defensor,Roma,1
4,798820,Artem Dovbyk,Atacante,Roma,1
5,848287,Evan Ndicka,Defensor,Roma,1
6,1134200,Wesley,Defensor,Roma,1


Eliminando a coluna "player_id"

In [236]:
gols = gols.drop(gols.columns[0], axis=1)

gols.head(3)

,name,position,team,gols
0,Matías Soulé,Atacante,Roma,3
1,Bryan Cristante,Meio-campo,Roma,1
2,Lorenzo Pellegrini,Meio-campo,Roma,1


Salvando as informações

In [237]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    gols.head(10).to_excel(writer, sheet_name="Artilharia", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Assistências

Criando uma cópia do DataFrame original

In [238]:
assistencias = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [239]:
game_dates = (
    assistencias
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [240]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [241]:
assistencias = (
    assistencias
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [242]:
assistencias = (
    assistencias
      .groupby(["player_id", "name", "position", "team"], as_index=False)["goal_assist"]
      .sum()
      .rename(columns={"goal_assist": "assistencias"})
      .sort_values(["team", "assistencias"], ascending=[True, False])
      .reset_index(drop=True))

Eliminando os jogadores que não deram nenhuma assistência

In [243]:
assistencias = assistencias[assistencias["assistencias"] > 0]

Removendo os valores duplicados

In [244]:
assistencias = assistencias.drop_duplicates()

### Resultado: Assistências

Ordenando do maior para o menor

In [245]:
assistencias = assistencias.sort_values("assistencias", ascending=False)

assistencias = assistencias.head(10)

assistencias

,player_id,name,position,team,assistencias
0,1082406,Matías Soulé,Atacante,Roma,2
1,555540,Lorenzo Pellegrini,Meio-campo,Roma,1
2,786259,Konstantinos Tsimikas,Defensor,Roma,1
3,798820,Artem Dovbyk,Atacante,Roma,1
4,893008,Zeki Çelik,Defensor,Roma,1
5,999231,Evan Ferguson,Atacante,Roma,1


Eliminando a coluna "player_id"

In [246]:
assistencias = assistencias.drop(assistencias.columns[0], axis=1)

assistencias.head(3)

,name,position,team,assistencias
0,Matías Soulé,Atacante,Roma,2
1,Lorenzo Pellegrini,Meio-campo,Roma,1
2,Konstantinos Tsimikas,Defensor,Roma,1


Salvando as informações

In [247]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    assistencias.head(10).to_excel(writer, sheet_name="Assistências", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Participação em gols

In [248]:
participacao_gols = df.copy()

Criando a coluna com a participação em gols

In [249]:
participacao_gols["participacoes"] = participacao_gols["goals"] + participacao_gols["goal_assist"]

participacao_gols.head(3)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals,participacoes
2931,2025-08-30,13981441,2025,Fora,1128530,Neil El Aynaoui,Meio-campo,Roma,0.0,1,0,0,0,0,0.0,0
2932,2025-08-30,13981441,2025,Fora,982778,Devyne Rensch,Defensor,Roma,0.0,1,0,0,0,0,0.0,0
2933,2025-08-30,13981441,2025,Fora,893008,Zeki Çelik,Defensor,Roma,6.9,8,0,0,0,0,0.0,0


Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [250]:
game_dates = (
    participacao_gols
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [251]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [252]:
participacao_gols = (
    participacao_gols
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [253]:
participacao_gols = (
    participacao_gols
      .groupby(["player_id", "name", "position", "team"], as_index=False)["participacoes"]
      .sum()
      .sort_values(["team", "participacoes"], ascending=[True, False])
      .reset_index(drop=True))


Removendo os jogadores sem participação em gols

In [254]:
participacao_gols = participacao_gols[participacao_gols["participacoes"] > 0]

participacao_gols

,player_id,name,position,team,participacoes
0,1082406,Matías Soulé,Atacante,Roma,5
1,555540,Lorenzo Pellegrini,Meio-campo,Roma,2
2,798820,Artem Dovbyk,Atacante,Roma,2
3,186855,Bryan Cristante,Meio-campo,Roma,1
4,611210,Gianluca Mancini,Defensor,Roma,1
5,786259,Konstantinos Tsimikas,Defensor,Roma,1
6,848287,Evan Ndicka,Defensor,Roma,1
7,893008,Zeki Çelik,Defensor,Roma,1
8,999231,Evan Ferguson,Atacante,Roma,1
9,1134200,Wesley,Defensor,Roma,1


### Resultado: Participação em gols

Ordenando do maior para o menor

In [255]:
participacao_gols = participacao_gols.sort_values("participacoes", ascending=False)

participacao_gols = participacao_gols.head(10)

participacao_gols

,player_id,name,position,team,participacoes
0,1082406,Matías Soulé,Atacante,Roma,5
1,555540,Lorenzo Pellegrini,Meio-campo,Roma,2
2,798820,Artem Dovbyk,Atacante,Roma,2
3,186855,Bryan Cristante,Meio-campo,Roma,1
4,611210,Gianluca Mancini,Defensor,Roma,1
5,786259,Konstantinos Tsimikas,Defensor,Roma,1
6,848287,Evan Ndicka,Defensor,Roma,1
7,893008,Zeki Çelik,Defensor,Roma,1
8,999231,Evan Ferguson,Atacante,Roma,1
9,1134200,Wesley,Defensor,Roma,1


Eliminando a coluna "player_id"

In [256]:
participacao_gols = participacao_gols.drop(participacao_gols.columns[0], axis=1)

participacao_gols.head(3)

,name,position,team,participacoes
0,Matías Soulé,Atacante,Roma,5
1,Lorenzo Pellegrini,Meio-campo,Roma,2
2,Artem Dovbyk,Atacante,Roma,2


Salvando as informações

In [257]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    participacao_gols.head(10).to_excel(writer, sheet_name="Participação em Gols", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Rating Total

In [258]:
rating = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [259]:
game_dates = (
    rating
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [260]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [261]:
rating = (
    rating
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [262]:
rating = (
    rating
      .groupby(["player_id", "name", "position", "team"], as_index=False)["rating"]
      .sum()
      .rename(columns={"rating": "rating_total"})
      .sort_values(["team", "rating_total"], ascending=[True, False])
      .reset_index(drop=True))

Montando o top 5 de cada clube

In [263]:
rating_casa = rating[rating["team"] == clubes.casa].head(5)

rating_casa

,player_id,name,position,team,rating_total
0,1082406,Matías Soulé,Atacante,Roma,67.0
1,611210,Gianluca Mancini,Defensor,Roma,63.5
2,793986,Mile Svilar,Goleiro,Roma,63.3
3,974087,Manu Koné,Meio-campo,Roma,62.2
4,186855,Bryan Cristante,Meio-campo,Roma,62.1


In [264]:
rating_fora = rating[rating["team"] == clubes.fora].head(5)

rating_fora

,player_id,name,position,team,rating_total


Unificando as informações em um único DataFrame

In [265]:
rating = pd.concat([rating_casa, rating_fora], ignore_index=True).sort_values("rating_total", ascending=False)

rating

,player_id,name,position,team,rating_total
0,1082406,Matías Soulé,Atacante,Roma,67.0
1,611210,Gianluca Mancini,Defensor,Roma,63.5
2,793986,Mile Svilar,Goleiro,Roma,63.3
3,974087,Manu Koné,Meio-campo,Roma,62.2
4,186855,Bryan Cristante,Meio-campo,Roma,62.1


### Resultado: Rating Total

Ordenando do maior para o menor

In [266]:
rating = rating[rating["rating_total"] > 0]
rating["rating_total"] = rating["rating_total"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

rating.head(10).style.hide(axis="index")

player_id,name,position,team,rating_total
1082406,Matías Soulé,Atacante,Roma,"67,00"
611210,Gianluca Mancini,Defensor,Roma,"63,50"
793986,Mile Svilar,Goleiro,Roma,"63,30"
974087,Manu Koné,Meio-campo,Roma,"62,20"
186855,Bryan Cristante,Meio-campo,Roma,"62,10"


Eliminando a coluna "player_id"

In [267]:
rating = rating.drop(rating.columns[0], axis=1)

rating

,name,position,team,rating_total
0,Matías Soulé,Atacante,Roma,"67,00"
1,Gianluca Mancini,Defensor,Roma,"63,50"
2,Mile Svilar,Goleiro,Roma,"63,30"
3,Manu Koné,Meio-campo,Roma,"62,20"
4,Bryan Cristante,Meio-campo,Roma,"62,10"


Salvando as informações

In [268]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    rating.head(10).to_excel(writer, sheet_name="Rating Total", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Rating Médio

Criando uma cópia do DataFrame original

In [269]:
rating_medio = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [270]:
game_dates = (
    rating_medio
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [271]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [272]:
rating_medio = (
    rating_medio
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [273]:
rating_medio = (
    rating_medio
      .groupby(["player_id", "name", "position", "team"], as_index=False)["rating"]
      .mean()
      .rename(columns={"rating": "rating_medio"})
      .sort_values(["team", "rating_medio"], ascending=[True, False])
      .reset_index(drop=True))

#### Criando a coluna de minutos jogados

Criando uma cópia do DataFrame original

In [274]:
minutos_jogados = df.copy()

Data do jogo por "time + jogo"

In [275]:
game_dates = (
    minutos_jogados
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [276]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [277]:
minutos_jogados = (
    minutos_jogados
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [278]:
minutos_jogados = (
    minutos_jogados
      .groupby(["player_id", "team"], as_index=False)["minutes_played"]
      .sum()
      .rename(columns={"minutes_played": "minutos_jogados"})
      .sort_values(["team", "minutos_jogados"], ascending=[True, False])
      .reset_index(drop=True))

Transformando os minutos jogados em inteiro

In [279]:
minutos_jogados["minutos_jogados"] = minutos_jogados["minutos_jogados"].astype(int)

minutos_jogados.head(3)

,player_id,team,minutos_jogados
0,793986,Roma,810
1,848287,Roma,756
2,611210,Roma,755


Unificando os dois DataFrames

In [280]:
rating_medio = rating_medio.merge(minutos_jogados, how="inner", on="player_id")

rating_medio.head(3)

,player_id,name,position,team_x,rating_medio,team_y,minutos_jogados
0,1082406,Matías Soulé,Atacante,Roma,7.444444,Roma,728
1,611210,Gianluca Mancini,Defensor,Roma,7.055556,Roma,755
2,793986,Mile Svilar,Goleiro,Roma,7.033333,Roma,810


Filtrando os jogadores com mais de 10% de participação nos jogos

In [281]:
rating_medio = rating_medio[rating_medio["minutos_jogados"] >= 90]

rating_medio.head(3)

,player_id,name,position,team_x,rating_medio,team_y,minutos_jogados
0,1082406,Matías Soulé,Atacante,Roma,7.444444,Roma,728
1,611210,Gianluca Mancini,Defensor,Roma,7.055556,Roma,755
2,793986,Mile Svilar,Goleiro,Roma,7.033333,Roma,810


Formantando o valor do rating médio com apenas duas casas decimais

In [282]:
rating_medio["rating_medio"] = rating_medio["rating_medio"].round(2)

rating_medio.head(3)

,player_id,name,position,team_x,rating_medio,team_y,minutos_jogados
0,1082406,Matías Soulé,Atacante,Roma,7.44,Roma,728
1,611210,Gianluca Mancini,Defensor,Roma,7.06,Roma,755
2,793986,Mile Svilar,Goleiro,Roma,7.03,Roma,810


### Resultado: Rating Médio

Ordenando do maior para o menor

In [283]:
rating_medio = rating_medio[["name", "position", "team_x", "rating_medio"]]
rating_medio = rating_medio.sort_values("rating_medio", ascending=False)
rating_medio["rating_medio"] = rating_medio["rating_medio"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

rating_medio.head(10).style.hide(axis="index")

name,position,team_x,rating_medio
Matías Soulé,Atacante,Roma,"7,44"
Gianluca Mancini,Defensor,Roma,"7,06"
Mile Svilar,Goleiro,Roma,"7,03"
Wesley,Defensor,Roma,"7,01"
Manu Koné,Meio-campo,Roma,"6,91"
Bryan Cristante,Meio-campo,Roma,"6,90"
Lorenzo Pellegrini,Meio-campo,Roma,"6,87"
Angeliño,Meio-campo,Roma,"6,86"
Evan Ndicka,Defensor,Roma,"6,84"
Paulo Dybala,Atacante,Roma,"6,80"


In [284]:
rating_medio = rating_medio.head(10)

Salvando as informações

In [285]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    rating_medio.head(10).to_excel(writer, sheet_name="Rating Médio", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Grandes Chances Criadas

Criando uma cópia do DataFrame principal

In [286]:
grandes_chances = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [287]:
game_dates = (
    grandes_chances
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [288]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [289]:
grandes_chances = (
    grandes_chances
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [290]:
grandes_chances.head(2)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals
0,2025-08-30,13981441,2025,Fora,1128530,Neil El Aynaoui,Meio-campo,Roma,0.0,1,0,0,0,0,0.0
1,2025-08-30,13981441,2025,Fora,982778,Devyne Rensch,Defensor,Roma,0.0,1,0,0,0,0,0.0


In [291]:
grandes_chances = (
    grandes_chances
      .groupby(["player_id", "name", "position", "team"], as_index=False)["big_chance_created"]
      .sum()
      .rename(columns={"big_chance_created": "grandes_chances"})
      .sort_values(["team", "grandes_chances"], ascending=[True, False])
      .reset_index(drop=True))

Eliminado os jogadores que não criaram nenhuma grande chance

In [292]:
grandes_chances = grandes_chances[grandes_chances["grandes_chances"] > 0]

grandes_chances.head(3)

,player_id,name,position,team,grandes_chances
0,1082406,Matías Soulé,Atacante,Roma,3
1,555540,Lorenzo Pellegrini,Meio-campo,Roma,2
2,974087,Manu Koné,Meio-campo,Roma,2


### Resultado: Grandes Chances Criadas

In [293]:
grandes_chances = grandes_chances.sort_values("grandes_chances", ascending=False)

grandes_chances.head(3)

,player_id,name,position,team,grandes_chances
0,1082406,Matías Soulé,Atacante,Roma,3
1,555540,Lorenzo Pellegrini,Meio-campo,Roma,2
2,974087,Manu Koné,Meio-campo,Roma,2


Eliminando a coluna "player_id"

In [294]:
grandes_chances = grandes_chances.drop(grandes_chances.columns[0], axis=1)

grandes_chances.head(3)

,name,position,team,grandes_chances
0,Matías Soulé,Atacante,Roma,3
1,Lorenzo Pellegrini,Meio-campo,Roma,2
2,Manu Koné,Meio-campo,Roma,2


In [295]:
grandes_chances = grandes_chances.head(10)

Salvando as informações

In [296]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    grandes_chances.head(10).to_excel(writer, sheet_name="Grandes Chances", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Passes chave

Criando uma cópia do DataFrame original

In [297]:
passes_chave = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [298]:
game_dates = (
    passes_chave
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [299]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [300]:
passes_chave = (
    passes_chave
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [301]:
passes_chave = (
    passes_chave
      .groupby(["player_id", "name", "position", "team"], as_index=False)["key_pass"]
      .sum()
      .rename(columns={"key_pass": "passes_chave"})
      .sort_values(["team", "passes_chave"], ascending=[True, False])
      .reset_index(drop=True))

Removendo os jogadores que não deram nenhum passe-chave

In [302]:
passes_chave = passes_chave[passes_chave["passes_chave"] > 0]

passes_chave.head(3)

,player_id,name,position,team,passes_chave
0,1082406,Matías Soulé,Atacante,Roma,17
1,555540,Lorenzo Pellegrini,Meio-campo,Roma,9
2,999231,Evan Ferguson,Atacante,Roma,8


### Resultado: Passes Chave

In [303]:
passes_chave = passes_chave.sort_values("passes_chave", ascending=False)

passes_chave.head(3)

,player_id,name,position,team,passes_chave
0,1082406,Matías Soulé,Atacante,Roma,17
1,555540,Lorenzo Pellegrini,Meio-campo,Roma,9
2,999231,Evan Ferguson,Atacante,Roma,8


Eliminando a coluna "player_id"

In [304]:
passes_chave = passes_chave.drop(passes_chave.columns[0], axis=1)

passes_chave.head(3)

,name,position,team,passes_chave
0,Matías Soulé,Atacante,Roma,17
1,Lorenzo Pellegrini,Meio-campo,Roma,9
2,Evan Ferguson,Atacante,Roma,8


In [305]:
passes_chave = passes_chave.head(10)

Salvando as informações

In [306]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    passes_chave.to_excel(writer, sheet_name="Passes Chave", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### XG Total

In [307]:
xg = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [308]:
game_dates = (
    xg
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [309]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [310]:
xg = (
    xg
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [311]:
xg = (
    xg
      .groupby(["player_id", "name", "position", "team"], as_index=False)["expected_goals"]
      .sum()
      .rename(columns={"expected_goals": "xg_total"})
      .sort_values(["team", "xg_total"], ascending=[True, False])
      .reset_index(drop=True))

Eliminando os valores igual a 0

In [312]:
xg = xg[xg["xg_total"] > 0]

xg.head(3)

,player_id,name,position,team,xg_total
0,1082406,Matías Soulé,Atacante,Roma,2.5049
1,798820,Artem Dovbyk,Atacante,Roma,1.8522
2,848287,Evan Ndicka,Defensor,Roma,0.9810


### Resultado: XG Total

In [313]:
xg = xg.sort_values("xg_total", ascending=False)
xg["xg_total"] = xg["xg_total"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

xg.head(3).style.hide(axis="index")

player_id,name,position,team,xg_total
1082406,Matías Soulé,Atacante,Roma,"2,50"
798820,Artem Dovbyk,Atacante,Roma,"1,85"
848287,Evan Ndicka,Defensor,Roma,"0,98"


Eliminando a coluna "player_id"

In [314]:
xg = xg.drop(xg.columns[0], axis=1)

xg.head(3)

,name,position,team,xg_total
0,Matías Soulé,Atacante,Roma,"2,50"
1,Artem Dovbyk,Atacante,Roma,"1,85"
2,Evan Ndicka,Defensor,Roma,"0,98"


In [315]:
xg = xg.head(10)

Salvando as informações

In [316]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    xg.to_excel(writer, sheet_name="xG Total", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### XG Médio


Criando uma cópia do DataFrame original

In [317]:
xg_medio = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [318]:
game_dates = (
    xg_medio
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [319]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [320]:
xg_medio = (
    xg_medio
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [321]:
xg_medio = (
    xg_medio
      .groupby(["player_id", "name", "position", "team"], as_index=False)["expected_goals"]
      .mean()
      .rename(columns={"expected_goals": "xg_medio"})
      .sort_values(["team", "xg_medio"], ascending=[True, False])
      .reset_index(drop=True))


Removendo os valores igual a 0

In [322]:
xg_medio = xg_medio[xg_medio["xg_medio"] > 0]

xg_medio.head(3)

,player_id,name,position,team,xg_medio
0,1082406,Matías Soulé,Atacante,Roma,0.278322
1,798820,Artem Dovbyk,Atacante,Roma,0.231525
2,256219,Paulo Dybala,Atacante,Roma,0.140160


### Resultado: XG Médio

In [323]:
xg_medio = xg_medio.sort_values("xg_medio", ascending=False)
xg_medio["xg_medio"] = xg_medio["xg_medio"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

xg_medio.head(3).style.hide(axis="index")

player_id,name,position,team,xg_medio
1082406,Matías Soulé,Atacante,Roma,"0,28"
798820,Artem Dovbyk,Atacante,Roma,"0,23"
256219,Paulo Dybala,Atacante,Roma,"0,14"


Eliminando a coluna "player_id"

In [324]:
xg_medio = xg_medio.drop(xg_medio.columns[0], axis=1)

xg_medio.head(3)

,name,position,team,xg_medio
0,Matías Soulé,Atacante,Roma,"0,28"
1,Artem Dovbyk,Atacante,Roma,"0,23"
2,Paulo Dybala,Atacante,Roma,"0,14"


In [325]:
xg_medio = xg_medio.head(10)

Salvando o arquivo

In [326]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    xg_medio.to_excel(writer, sheet_name="xG Médio", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


# Prompt

In [327]:
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn

# =========================================
# OPCIONAIS: deixar a string dos DFs sem cortes/quebras
# =========================================
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)  # evita wrap precoce

# =========================================
# FUNÇÃO: converte objeto em texto sem índice
# - Se for Series, vira DataFrame primeiro
# - Se não existir, retorna string vazia (evita quebrar o f-string)
# =========================================
def to_text_no_index(obj):
    if obj is None:
        return ""
    if isinstance(obj, pd.Series):
        obj = obj.to_frame()
    if isinstance(obj, pd.DataFrame):
        # justify="left" ajuda a manter colunas previsíveis
        return obj.to_string(index=False, justify="left")
    return str(obj)

# =========================================
# PEGAR TODOS OS DFs CITADOS E CONVERTER
# (se algum não existir no seu ambiente, vira "")
# =========================================
globals_dict = globals()
names = [
    "tabela", "tabela_recente", "tabela_tempo1", "tabela_tempo2",
    "tabela_completo", "df_cleans", "desempenho_casa", "desempenho_fora",
    "desempenho_confronto", "historico_recente", "mais_minutos", "gols", "assistencias",
    "participacao_gols", "rating", "rating_medio", "grandes_chances",
    "passes_chave", "xg", "xg_medio"
]

conv = {}
for n in names:
    conv[n] = to_text_no_index(globals_dict.get(n))

# =========================================
# AQUI ASSUMO QUE VOCÊ JÁ TEM O OBJETO `clubes`
# com atributos .casa, .fora, .campeonato, .data, .hora
# =========================================

texto = f"""[ PAPEL ] 

Atue como Designer e Editor de Dados Sênior. Crie uma apresentação tipo e-book para a Bet Dados no Gamma (tema: Coal).

[ CONTEXTO ]

Público leigo–intermediário; foco em clareza.

Use somente os dados fornecidos. Sem inferências, estimativas ou previsões.

Se um dado não existir, omite a seção.

Duelo no formato “Time A x Time B” (nunca “vs”).

Estética “big tech”: precisão, consistência, espaço em branco e alinhamento milimétrico.

[ ESTILO GLOBAL — CONTRASTE OBRIGATÓRIO ] Forçar cor do texto = #FFFFFF para qualquer elemento textual, sem exceções:

Aplica cor #FFFFFF com opacidade 100% para: títulos, subtítulos, parágrafos, bullets, textos de cartões, subtítulos/captions, notas, footers, botões, placeholders, links, números de ordenação (ex.: “01”, “02” no canto), legendas de gráficos, rótulos de eixos, rótulos de dados, títulos de tabela e células de tabela.

Proibido usar cinzas/“muted”, opacidade < 100% ou “secondary text”. Nada de “texto desabilitado” em cinza.

Se houver fundo claro/Imagem, coloque placa/overlay escuro (ex.: #0A0A0A 90–95%) antes de posicionar o texto.

Ícones/linhas divisórias podem usar #FFFFFF a 70–90% de opacidade; o texto nunca.

[ GRÁFICOS ]

Desligar cores padrão do tema.

Barras/fatias/linhas: usar apenas cinzas claros #F2F2F2, #EAEAEA, #E0E0E0, #D5D5D5. (Nunca cinzas escuros.)

Contorno das barras/fatias: #FFFFFF (80–100% opacidade) para separação.

Legenda, eixos e labels: #FFFFFF.

Grade discreta: #9E9E9E a 25–35% opacidade. Fundo do gráfico transparente.

Cores oficiais dos clubes apenas como acento (borda/marker), mantendo contraste AA.

[ IDENTIDADE E IMAGENS ]

Escudos oficiais (sem distorção nem marca d’água). Se não houver, usar placeholder neutro.

Proibição absoluta de imagens com marca d’água.

Paleta do deck: #001636 como acento + neutros escuros de fundo + dados em cinzas claros.

[ TAREFA — SLIDES ] (omita os que não tiverem dados)

Capa — Título “[Time A x Time B] | Sub “[Data] — [Hora]”; chips de mandante/visitante e fase; troféu oficial como hero; overlay escuro se preciso.

Publicidade Bet Dados (obrigatório) — texto, CTA “Assinar agora” → https://pay.cakto.com.br/35ww97h_563195 e placeholder de logo.

Na publicidade, frise que a Bet Dados tem dados de mais de 90 mil jogos, mais de 140 campeonatos, competições de mais de 40 países,
mais de 4.200 clubes e mais de 115 mil jogadores. Ressalte também que a Bet Dados é um grupo no Telegram, onde são enviadas análises
dos principais jogos do dia, além de análises especiais com dados de ligas, clubes e/ou jogadores.

Ficha do Jogo — competição, temporada, data/hora, estádio/mandante.

Panorama das Equipes (cards lado a lado, 2–4 linhas por equipe, texto branco).

Indicadores-Chave (tabela/tiles).

Distribuição de Gols por Minutagem.

Ofensivo vs Defensivo.

Bolas Paradas.

Disciplina.

Tendências Temporais.

Head-to-Head (se houver).

Glossário Rápido (somente termos presentes).

Notas Metodológicas (origem/janela/limitações; “sem prognósticos”).

Slide Final (QR/CTA).

[ REGRAS DE CONTEÚDO ]

Sem previsões/odds.

Em hipótese alguma utilize imagens com marca d'água.

Nomes dos clubes exatamente como informados (acentos e caixa).

Percentuais com até 2 casas decimais; unidades PT-BR.

[ ENTRADAS ] Duelo, competição, temporada, data/hora e todas as estatísticas fornecidas (médias, totais, séries, rankings etc.).

[ AUDITORIA DE CONTRASTE (passo obrigatório no final) ]

Varra o deck e liste qualquer elemento cujo texto NÃO esteja #FFFFFF (inclui textos de cartões, subtítulos/captions, numeradores “01/02/03/04”, descrições, botões, rodapés e legendas).

Corrija automaticamente para #FFFFFF e reporte “Corrigido: [elementos]”. Se algo não puder ser alterado, substitua por placa/overlay escuro até garantir contraste AA.

[ CRITÉRIOS DE QUALIDADE ]

Fidelidade 100% aos dados.

Texto 100% branco em todo o deck; gráficos em cinzas claros; contraste AA mínimo.

Coesão (grid, margens, alinhamentos).

Didática (1 frase objetiva por gráfico).

Conformidade de marca (escudos/nome de clubes corretos; zero marca d’água).

[ CHECKLIST FINAL ]

Todo texto branco #FFFFFF (inclui cartões, captions e numeradores “01/02/03/…”).

Séries de gráficos somente #F2F2F2–#D5D5D5 + contorno branco; legenda/eixos brancos.

Sem imagens com marca d’água; escudos/taça oficiais.

“x” no duelo; nomes dos clubes fiéis.

Seções sem dados omitidas; nenhum número inventado.

Exportável para PDF com contraste perfeito.

Não trazer imagens com marca d'água.

Os dados e as estatísticas são de um levantamento da Bet Dados.

NÃO precisa informar a temporada.

Reforçando: NÃO FAÇA NENHUMA PREVISÃO PARA A PARTIDA!

Nos gráficos, SEMPRE ative o rótulo de dados (mostrar valores).

Importante: a Bet Dados NÃO oferece dados em tempo real. Não coloque isso na apresentação. Não coloque NENHUM ponto negativo da Bet Dados.

Não tenho QR Code, então não peça para o usuário scannear um QR Code. Peça apenas para clicar no botão e assinar a Bet Dados. Explique que, após o pagamento,
o link de acesso ao grupo no Telegram será enviado por e-mail para o usuário.

COLOQUE IMAGEM APENAS NA CAPA E NA CONTRACAPA, NÃO COLOQUE MAIS NENHUMA IMAGEM NA APRESENTAÇÃO!

[ DADOS DA PARTIDA ]

Duelo: {clubes.confronto}

Competição: {clubes.campeonato}

Data: {clubes.data}

Hora: {clubes.hora}

Segue abaixo as estatísticas para a apresentação:

Temporada atual

{conv['tabela']}

Todas as informações abaixo são baseadas nos últimos 10 jogos do(a) {clubes.casa}. Deixe isso explícito e coloque um asterisco em todos os slides informando que os dados são das últimas 10 partidas do(a) {clubes.casa}.

Desempenho recente

{conv['tabela_recente']}

Primeiro tempo

{conv['tabela_tempo1']}

Segundo tempo

{conv['tabela_tempo2']}

Desempenho nos 90 minutos

{conv['tabela_completo']}

Clean sheets e clean goals
{conv['df_cleans']}

Desempenho em casa do(a) {clubes.casa}
{conv['desempenho_casa']}

Confronto direto desde 2020
{conv['desempenho_confronto']}

Confronto direto - últimos 10 jogos
{conv['historico_recente']}

Mais minutos

{conv['mais_minutos']}

Artilharia

{conv['gols']}

Assistências

{conv['assistencias']}

Participação em gols

{conv['participacao_gols']}

Rating total

{conv['rating']}

Rating médio

{conv['rating_medio']}

Grandes chances criadas

{conv['grandes_chances']}

Passes chave

{conv['passes_chave']}

xG total

{conv['xg']}

xG médio

{conv['xg_medio']}
"""

In [328]:
import os

# === Descobre a pasta do projeto ===
try:
    # pasta onde este script .py está salvo
    pasta_projeto = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # fallback: pasta atual (para notebook/console)
    pasta_projeto = os.getcwd()

# === Monta o caminho do arquivo dentro da pasta do projeto ===
caminho_arquivo = os.path.join(pasta_projeto, "prompt.txt")

# === Salva como .txt (UTF-8) ===
with open(caminho_arquivo, "w", encoding="utf-8", newline="\n") as f:
    f.write(texto)

print(f"Arquivo .txt salvo com sucesso em: {caminho_arquivo}")


Arquivo .txt salvo com sucesso em: c:\Pessoal\Data_science\Projetos\sofascore\analises\prompt.txt


# Finalizando

In [329]:
print("Finalizado")

Finalizado
